In [1]:
exp_name = 'codegen-350M-mon_fullFTthestack_pass100x200_python'
sf = f'src/{exp_name}.jsonl'
td = f'tgt/{exp_name.replace("-", "")}'
!python convert_to_pre_eval.py --source_file $sf --target_dir $td


Done.



100%|██████████| 7200/7200 [00:00<00:00, 899850.68jsonline/s]


In [2]:
import subprocess
sf = f'src/{exp_name}.jsonl'
td = f'tgt/{exp_name.replace("-", "")}'
eval_cmd = f"podman run --rm --network none -v ./{td}:/{td}:rw multipl-e-eval --dir /{td} --output-dir /{td} --recursive"
subprocess.run(eval_cmd, shell=True)


CompletedProcess(args='podman run --rm --network none -v ./tgt/codegen350Mmon_fullFTthestack_pass100x200_python:/tgt/codegen350Mmon_fullFTthestack_pass100x200_python:rw multipl-e-eval --dir /tgt/codegen350Mmon_fullFTthestack_pass100x200_python --output-dir /tgt/codegen350Mmon_fullFTthestack_pass100x200_python --recursive', returncode=0)

In [3]:
sf = f'src/{exp_name}.jsonl'
td = f'tgt/{exp_name.replace("-", "")}'
target_dir = f'{td}'
output_file = f'res/{exp_name}_results.json'
!python pass_k.py $target_dir --output $output_file

Dataset,Pass@k,Estimate,NumProblems,MinCompletions,MaxCompletions
Reading results from tgt/codegen350Mmon_fullFTthestack_pass100x200_python...
And saving results to res/codegen-350M-mon_fullFTthestack_pass100x200_python_results.json...
Read 8 results
Sample result: {'pass@1': 0.0, 'pass@10': 0.0, 'pass@100': 0.0, 'n': 400, 'temperature': 0.8}
codegen350Mmon_fullFTthestack_pass100x200_python,10,0.060441859070631584,8,200,1600
codegen350Mmon_fullFTthestack_pass100x200_python,100,0.18717958209195942,8,200,1600
{'pass@10': {'dataset': 'codegen350Mmon_fullFTthestack_pass100x200_python',
             'estimate': 0.060441859070631584,
             'max_completions': 1600,
             'min_completions': 200,
             'num_problems': 8,
             'pass@k': 10},
 'pass@100': {'dataset': 'codegen350Mmon_fullFTthestack_pass100x200_python',
              'estimate': 0.18717958209195942,
              'max_completions': 1600,
              'min_completions': 200,
              'num_problems'